# Test imputation on untyped SNPs chrom 20

In [5]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using StatsBase
using CodecZlib
using ProgressMeter
using JLD2, FileIO, JLSO
using BenchmarkTools
using GroupSlices
using TimerOutputs
using LinearAlgebra

BLAS.set_num_threads(1)

┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1278


# MendelImpute error rate (window-window intersection)

In [2]:
Threads.nthreads()

8

In [3]:
# 8 threads
Random.seed!(2020)
d       = 1000
tgtfile = "target.chr20.typedOnly.maf0.01.masked.vcf.gz"
reffile = "ref.chr20.maxd$d.maf0.01.excludeTarget.jlso"
outfile = "mendel.chr20.imputed.target.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, max_d = d,
    dynamic_programming = false);

# import imputed result and compare with true
# X_mendel = convert_gt(Float32, outfile)
# X_complete = convert_gt(Float32, "target.chr20.full.vcf.gz")
# n, p = size(X_mendel)
# println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:26
Computing optimal haplotypes...100%|████████████████████| Time: 0:01:10
Phasing...100%|█████████████████████████████████████████| Time: 0:00:47
Writing to file...100%|█████████████████████████████████| Time: 0:00:11


Total windows = 3252, averaging ~ 510 unique haplotypes per window.

Timings: 
    Data import                     = 60.2973 seconds
        import target data             = 28.3326 seconds
        import compressed haplotypes   = 31.9647 seconds
    Computing haplotype pair        = 70.847 seconds
        BLAS3 mul! to get M and N      = 1.08111 seconds per thread
        haplopair search               = 60.2235 seconds per thread
        initializing missing           = 1.38959 seconds per thread
        allocating and viewing         = 0.0989429 seconds per thread
        index conversion               = 0.0807914 seconds per thread
    Phasing by win-win intersection = 47.3326 seconds
        Window-by-window intersection  = 40.4241 seconds per thread
        Breakpoint search              = 1.00163 seconds per thread
        Recording result               = 2.8335 seconds per thread
    Imputation                     = 17.0572 seconds
        Imputing missing               = 5.131

In [25]:
;ls -alh phaseinfo.jlso

-rw-r--r--  1 biona001  staff    43M Aug 20 13:15 phaseinfo.jlso


In [26]:
;ls -alh mendel.chr20.imputed.target.vcf.gz

-rw-r--r--  1 biona001  staff    58M Aug 20 13:10 mendel.chr20.imputed.target.vcf.gz


In [23]:
Base.summarysize(ph[1]) # there are 1000 of these

173312

In [24]:
ph[1]

HaplotypeMosaicPair(HaplotypeMosaic(882742, [1, 106, 172, 291, 471, 783, 830, 1143, 1264, 1458  …  881198, 881429, 881606, 881694, 881694, 881753, 882002, 882085, 882187, 882407], [1, 1, 2, 3, 4, 4, 5, 6, 6, 7  …  3244, 3245, 3246, 3247, 3247, 3248, 3249, 3250, 3251, 3252], [10, 23, 42, 16, 24, 36, 589, 601, 12, 26  …  17, 15, 15, 17, 17, 370, 23, 20, 14, 16]), HaplotypeMosaic(882742, [1, 172, 291, 471, 830, 1143, 1458, 1917, 2186, 2330  …  880964, 881198, 881429, 881606, 881694, 881753, 882002, 882085, 882187, 882407], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  3243, 3244, 3245, 3246, 3247, 3248, 3249, 3250, 3251, 3252], [15, 10, 12, 16, 7, 949, 2, 3, 1, 1  …  19, 13, 2, 2, 9, 13, 1, 4, 1, 2126]))

In [14]:
;open .

In [18]:
JLSO.save("phaseinfo.jlso", :ph => ph, format=:julia_serialize, 
    compression=:gzip)

In [ ]:
saved_ph = JLSO.load("phaseinfo.jlso")[:ph]

# Profile code

In [3]:
using Profile
using ProfileView
Profile.init(n = 10^7, delay = 0.01) # 1 backtrace per 0.01 second

In [4]:
# warmup on small problem
cd("/Users/biona001/.julia/dev/MendelImpute/simulation")
Random.seed!(2020)
d       = 1000
tgtfile = "./compare2/target.typedOnly.maf0.01.masked.vcf.gz"
reffile = "./compare2/ref.excludeTarget.maxd$d.jlso"
outfile = "./compare2/mendel.imputed.vcf.gz"
@profview phase(tgtfile, reffile, outfile = outfile, max_d = d,
    dynamic_programming = false);

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:18
Computing optimal haplotypes...100%|████████████████████| Time: 0:00:09
Writing to file...100%|█████████████████████████████████| Time: 0:00:17


Total windows = 64, averaging ~ 692 unique haplotypes per window.

Timings: 
    Data import                     = 40.6077 seconds
    Computing haplotype pair        = 20.0551 seconds
        BLAS3 mul! to get M and N      = 0.215178 seconds per thread
        haplopair search               = 2.63116 seconds per thread
        initializing missing           = 0.324441 seconds per thread
        allocating and viewing         = 0.0415098 seconds per thread
        index conversion               = 0.000730224 seconds per thread
    Phasing by win-win intersection = 11.1837 seconds
        Window-by-window intersection  = 0.0532841 seconds per thread
        Breakpoint search              = 0.102382 seconds per thread
        Recording result               = 0.00478878 seconds per thread
    Imputation                      = 18.7042 seconds



In [5]:
# profile real problem
cd("/Users/biona001/.julia/dev/MendelImpute/data/HRC")
Random.seed!(2020)
d       = 1000
tgtfile = "target.chr20.typedOnly.maf0.01.masked.vcf.gz"
reffile = "ref.chr20.maxd$d.maf0.01.excludeTarget.jlso"
outfile = "mendel.chr20.imputed.target.vcf.gz"
@profview phase(tgtfile, reffile, outfile = outfile, max_d = d,
    dynamic_programming = false);

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:01:42
Computing optimal haplotypes...100%|████████████████████| Time: 0:02:22
Phasing...100%|█████████████████████████████████████████| Time: 0:01:32
Writing to file...100%|█████████████████████████████████| Time: 0:02:18


Total windows = 3252, averaging ~ 510 unique haplotypes per window.

Timings: 
    Data import                     = 204.957 seconds
    Computing haplotype pair        = 142.399 seconds
        BLAS3 mul! to get M and N      = 2.35351 seconds per thread
        haplopair search               = 126.861 seconds per thread
        initializing missing           = 3.4539 seconds per thread
        allocating and viewing         = 0.17913 seconds per thread
        index conversion               = 0.133923 seconds per thread
    Phasing by win-win intersection = 92.7697 seconds
        Window-by-window intersection  = 74.2154 seconds per thread
        Breakpoint search              = 2.20214 seconds per thread
        Recording result               = 7.43114 seconds per thread
    Imputation                      = 172.38 seconds

/Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl, handle_deserialize: line 824
/Users/julia/buil